# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
 from pymongo import MongoClient
# client = MongoClient("")
dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
lista = db.list_collections()

In [3]:
cursor = db.ejercicios.find({"name":"Babelgum"})
for i,name in enumerate(cursor):
    print(name["name"])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [4]:
from pymongo import ASCENDING, DESCENDING
employes= list(db.ejercicios.find( {"number_of_employees": { "$gte": 5000}}).sort( [("number_of_employees" ,DESCENDING )] ).limit(20))

for i,name in enumerate(employes):
    print(name["name"])

Siemens
IBM
Toyota
PayPal
Nippon Telegraph and Telephone Corporation
Samsung Electronics
Accenture
Tata Consultancy Services
Flextronics International
Safeway
Sony
LG
Ford
Boeing
Digital Equipment Corporation
Nokia
MItsubishi Electric
MItsubishi Electric
Comcast
Bertelsmann


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [5]:
query = {"$and":[{"founded_year":{"$gte":2000}},{"founded_year":{ "$lte": 2005}}]}

db.ejercicios.find(query).count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


3734

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [6]:
# Your Code
query = {"$and":[{"ipo.valuation_amount":{"$gte":100000000}},{"founded_year":{ "$lte": 2010}}]}
ipoValue_Amount = db.ejercicios.find(query)
print(ipoValue_Amount.count())
for index, value in enumerate(ipoValue_Amount) :
    value["name"],value["ipo"]

42


/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [7]:
# Your Code
query = {"$and":[{"number_of_employees":{"$lte":1000}},{"founded_year":{ "$lte": 2005}}]}
Allcompany = db.ejercicios.find(query) .sort( [("number_of_employees" ,DESCENDING )] ).limit(10)
for index, value in enumerate(Allcompany) :
    print(value["name"])

Akamai Technologies
Omniture
Yodle
Sonus Networks
Telenav
Antal International
SK Net Service Company Ltd
Gumtree
FANUC Robotics America
OC'LIANE


### 6. All the companies that don't include the `partners` field.

In [8]:
# Your Code
query = {"partners":{ "$exists": False }}
partners = db.ejercicios.find(query).count()
partners


/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


0

### 7. All the companies that have a null type of value on the `category_code` field.

In [9]:
# Your Code
query ={"category_code": None}
category_code = db.ejercicios.find(query).count()
category_code

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


2751

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [10]:
query = {"$and":[{"number_of_employees": {"$gte":100}},{"number_of_employees":{"$lte":1000}}]}
companis =db.ejercicios.find((query),{"name":1,"number_of_employees":1})

for index, value in enumerate(companis) :
    value["name"],value["number_of_employees"]
companis.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


942

### 9. Order all the companies by their IPO price in a descending order.

In [11]:
IPO = db.ejercicios.find().sort([("ipo.price", DESCENDING)])
IPO.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


18801

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [12]:
top10 = db.ejercicios.find().sort([("number_of_employees", DESCENDING)]).limit(10)
for index, value in enumerate(top10) :
    print(value["name"])

Siemens
IBM
Toyota
PayPal
Nippon Telegraph and Telephone Corporation
Samsung Electronics
Accenture
Tata Consultancy Services
Flextronics International
Safeway


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [13]:
query = {"founded_month": {"$gt": 6}}
seconSemestre = db.ejercicios.find(query).limit(1000)
seconSemestre.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


3329

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [14]:
query = {"$and":[{"founded_year": {"$lt":2000}},{"acquisition.price_amount":{"$gte":1000000}}]}
AllCompani = db.ejercicios.find(query)
AllCompani.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


224

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [16]:
query ={"acquisition.acquired_year": {"$gt":2010}}
CompaniAf2010 = db.ejercicios.find(query).sort("acquisition.price_amount")
print(CompaniAf2010.count())
for index, value in enumerate(CompaniAf2010) :
     value ["name"], value ["acquisition"]

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


736


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [19]:
# Your Code
companiesFounded_year = db.ejercicios.find().sort([("founded_year", DESCENDING)]).limit(50)
for index, value in enumerate(companiesFounded_year) :
     value["name"], value["founded_year"]
        
companiesFounded_year.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


18801

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [24]:
query = {"$and":[{"founded_month":1},{"founded_day":{"$lte":7}}]}
Founded7Days = db.ejercicios.find(query).sort([("acquisition.price_amount", DESCENDING)]).limit(10)
Founded7Days.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


795

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [26]:
query = {"$and":[{"category_code": "web"}, {"number_of_employees":{"$gt":4000}}]}
CompanyWeb =db.ejercicios.find(query).sort([("number_of_employees", ASCENDING)])
CompanyWeb.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


9

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [29]:
query = {"$and" :[{"acquisition.price_amount":{"$gt":10000000}},{"price_currency_code": "EUR"}]}
CompanyAmount10m =db.ejercicios.find(query)

CompanyAmount10m.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


0

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [32]:
query = {"$and":[{"acquisition.acquired_month":{"$lt":6}}]}
CompanyPrimerTrimestre =db.ejercicios.find(query)

for index, value in enumerate(CompanyPrimerTrimestre) :
     value["name"], value["acquisition"]
CompanyPrimerTrimestre.count()

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


1193

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [35]:
query = {"$and":[{"founded_year":{"$gte":2000}},{"founded_year":{ "$lte": 2010}},
                 {"acquisition.acquired_year":{"$lt":2011}}]}
AllCompanys2000_2010NoAcqu2011 = db.ejercicios.find(query).count()
AllCompanys2000_2010NoAcqu2011

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


713

### 20. All the companies that have been 'deadpooled' after the third year.

In [38]:
AllCompanyDeadbef3year = db.ejercicios.find({"deadpooled_year":{"$lt":3}}).count()
AllCompanyDeadbef3year

/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2